In [1]:
import torch
import torch
import numpy as np
from gensim.models import Word2Vec

In [2]:
import gensim.downloader as api

tokenizer = api.load("glove-wiki-gigaword-300")

In [42]:
def embed_sentence(sentence, tokenizer):
    embeddings = []
    for word in sentence.split():
        word = word.lower()
        if not word in tokenizer: continue
        
        embeddings.append(tokenizer[word])
    
    if len(embeddings) == 0: # bug fix for padding function (we need to asssure at least one element)
        embeddings = np.zeros(shape=(1,tokenizer.vector_size))
    return torch.tensor(np.asarray(embeddings))

In [43]:
from pathlib import Path
import json

# read all sentences
sentences = []
for item in Path("training").iterdir():
    if not item.is_file(): continue 
    if not item.suffix == ".json": continue
    
    # load data
    dialog = json.load(open(item, "r"))
    sentences += [exchange["text"] for exchange in dialog]

In [44]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_sequence

# stacking sequences
embedded_sentences = [embed_sentence(sentence, tokenizer) for sentence in sentences]

# padding sequences so that they have same dimension
padded_sequences = pad_sequence(embedded_sentences, batch_first=True)

# packing sequences
lengths = [len(sentence.split()) for sentence in sentences]
packed_sequences = pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)

aqui tem que printar...


In [ ]:
# GPT
# class LSTMClassifier(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(LSTMClassifier, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, x, lengths):
#         # Pack the sequences
#         packed_input = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)

#         # Pass through LSTM
#         lstm_out, _ = self.lstm(packed_input)

#         # Unpack the sequences (if needed)
#         lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True)

#         # Take the output from the last time step
#         lstm_out = lstm_out[:, -1, :]

#         # Fully connected layer
#         output = self.fc(lstm_out)
#         return output
